3.1 Написать функцию формирования порождающей и проверочной матриц кода Хэмминга (𝟐𝒓 − 𝟏, 𝟐𝒓 − 𝒓 − 𝟏, 𝟑) на основе параметра 𝒓, а также таблицы синдромов для всех однократных ошибок.

In [9]:
import random

def identity_matrix(size):
    return [[1 if i == j else 0 for j in range(size)] for i in range(size)]

def combine_horizontally(matrix_a, matrix_b):
    return [a_row + b_row for a_row, b_row in zip(matrix_a, matrix_b)]

def combine_vertically(matrix_a, matrix_b):
    return matrix_a + matrix_b

def generate_combinations(n, k):
    combinations = []
    for i in range(1 << n):
        binary = list(map(int, f"{i:0{n}b}"))
        if sum(binary) == k:
            combinations.append(binary)
    return combinations

def create_hamming_matrix(r):
    n = 2 ** r - 1
    k = n - r
    identity = identity_matrix(k)
    parity_part = generate_combinations(n, k)
    return combine_horizontally(identity, parity_part[:k])

def create_parity_check_matrix(r):
    n = 2 ** r - 1
    k = n - r
    parity_part = generate_combinations(n, k)
    identity = identity_matrix(n - k)
    return combine_vertically(parity_part[:n - k], identity)

def multiply_vector_by_matrix(vector, matrix):
    return [sum(v * m for v, m in zip(vector, col)) % 2 for col in zip(*matrix)]

def create_syndrome_dict(h_matrix):
    syndrome_dict = {}
    for i in range(len(h_matrix[0])):
        error_vector = [0] * len(h_matrix[0])
        error_vector[i] = 1
        syndrome = multiply_vector_by_matrix(error_vector, h_matrix)
        syndrome_dict[tuple(syndrome)] = error_vector
    return syndrome_dict

def generate_extended_hamming_matrix(r):
    hamming_matrix = create_hamming_matrix(r)
    for row in hamming_matrix:
        row.append(sum(row) % 2)
    return hamming_matrix

def generate_extended_parity_matrix(r):
    parity_matrix = create_parity_check_matrix(r)
    extra_row = [1] * len(parity_matrix[0])
    for row in parity_matrix:
        row.append(0)
    parity_matrix.append(extra_row)
    return parity_matrix

def create_error_vector(size, errors):
    if errors > size:
        errors = size  # ограничим количество ошибок размером вектора
    vector = [0] * size
    for index in random.sample(range(size), errors):
        vector[index] = 1
    return vector

def fix_errors(h_matrix, received_word):
    syndrome = multiply_vector_by_matrix(received_word, h_matrix)
    syndrome_dict = create_syndrome_dict(h_matrix)
    if tuple(syndrome) in syndrome_dict:
        error_vector = syndrome_dict[tuple(syndrome)]
        corrected_word = [(bit + error) % 2 for bit, error in zip(received_word, error_vector)]
        return corrected_word, syndrome
    return received_word, syndrome

def analyze_hamming_code(r, extended=False):
    if extended:
        g_matrix = generate_extended_hamming_matrix(r)
        h_matrix = generate_extended_parity_matrix(r)
        error_limit = 4
        print("\nАнализ расширенного кода Хэмминга")
    else:
        g_matrix = create_hamming_matrix(r)
        h_matrix = create_parity_check_matrix(r)
        error_limit = 3
        print("\nАнализ стандартного кода Хэмминга")

    print("\nПорождающая матрица G:")
    for row in g_matrix:
        print(row)

    print("\nПроверочная матрица H:")
    for row in h_matrix:
        print(row)

    random_codeword = random.choice([list(col) for col in zip(*g_matrix)])
    print("\nСгенерированное кодовое слово:")
    print(random_codeword)

    for errors in range(1, error_limit + 1):
        print(f"\nПроверка для {errors} ошибок:")
        error_vector = create_error_vector(len(random_codeword), errors)
        print(f"Вектор ошибок: {error_vector}")

        received_word = [(bit + error) % 2 for bit, error in zip(random_codeword, error_vector)]
        print(f"Полученное слово с ошибками: {received_word}")

        corrected_word, syndrome = fix_errors(h_matrix, received_word)
        print(f"Синдром: {syndrome}")
        print(f"Исправленное слово: {corrected_word}")

        final_syndrome = multiply_vector_by_matrix(corrected_word, h_matrix)
        print(f"Синдром после коррекции: {final_syndrome}")

# Анализ стандартного и расширенного кодов Хэмминга
for r_value in [2, 3, 4]:
    analyze_hamming_code(r_value)
    analyze_hamming_code(r_value, extended=True)



Анализ стандартного кода Хэмминга

Порождающая матрица G:
[1, 0, 0, 1]

Проверочная матрица H:
[0, 0, 1]
[0, 1, 0]
[1, 0]
[0, 1]

Сгенерированное кодовое слово:
[1]

Проверка для 1 ошибок:
Вектор ошибок: [1]
Полученное слово с ошибками: [0]
Синдром: [0, 0]
Исправленное слово: [1]
Синдром после коррекции: [0, 0]

Проверка для 2 ошибок:
Вектор ошибок: [1]
Полученное слово с ошибками: [0]
Синдром: [0, 0]
Исправленное слово: [1]
Синдром после коррекции: [0, 0]

Проверка для 3 ошибок:
Вектор ошибок: [1]
Полученное слово с ошибками: [0]
Синдром: [0, 0]
Исправленное слово: [1]
Синдром после коррекции: [0, 0]

Анализ расширенного кода Хэмминга

Порождающая матрица G:
[1, 0, 0, 1, 0]

Проверочная матрица H:
[0, 0, 1, 0]
[0, 1, 0, 0]
[1, 0, 0]
[0, 1, 0]
[1, 1, 1]

Сгенерированное кодовое слово:
[0]

Проверка для 1 ошибок:
Вектор ошибок: [1]
Полученное слово с ошибками: [1]
Синдром: [0, 0, 1]
Исправленное слово: [0]
Синдром после коррекции: [0, 0, 0]

Проверка для 2 ошибок:
Вектор ошибок: [1]
По